In [2]:
import pandas as pd
import numpy as np
import re

#1️ Đọc dữ liệu từ file CSV
file_path = "patient_heart_rate.csv"
df = pd.read_csv(file_path, header=None, on_bad_lines='skip') # Skip lines with errors

In [4]:
#2️ Đặt tên cột cho dataset
# Get the current number of columns in the DataFrame
num_cols = df.shape[1]

# Create a list of column names with the correct length
# Assuming the first column was unnamed and you want to name it 'ID'
df.columns = ["ID"] + ["Name", "Age", "Weight", "HR1", "HR2", "HR3", "HR4", "HR5", "HR6"]

#Or if you don't know the name of the columns, you can auto-generate them
#df.columns = [f"Col{i}" for i in range(num_cols)]

# Now you can assign the new column names
#df.columns = new_column_names

In [5]:
#3️ Tách cột Name thành Firstname và Lastname
df[['Firstname', 'Lastname']] = df['Name'].str.split(expand=True)
df.drop(columns=['Name'], inplace=True)

In [7]:
#4️ Chuyển đổi đơn vị Weight về kg
def convert_weight(value):
    if "lbs" in str(value):
        return round(float(str(value).replace("lbs", "")) * 0.453592, 2)  # lbs -> kg
    # Attempt to extract numeric value using regex and handle potential errors
    try:
        # Extract numeric part of the string using regex
        numeric_value = float(re.findall(r'\d+', str(value))[0])
        return numeric_value
    except (IndexError, ValueError):
        # Handle cases where numeric value extraction fails
        print(f"Warning: Could not convert value: {value}. Returning NaN.")
        return np.nan  # Replace with NaN for invalid values
df["Weight"] = df["Weight"].apply(convert_weight)

In [8]:
#5️ Xóa dòng trống và dòng trùng lặp
df.dropna(how="all", inplace=True)
df.drop_duplicates(inplace=True)

In [9]:
#6️ Xử lý ký tự không phải ASCII
def remove_non_ascii(text):
    return re.sub(r'[^\x00-\x7F]+', '', str(text))
df["Firstname"] = df["Firstname"].apply(remove_non_ascii)
df["Lastname"] = df["Lastname"].apply(remove_non_ascii)

In [11]:
# 7. Xử lý dữ liệu bị thiếu
# Thay thế giá trị thiếu ở cột "Age" bằng giá trị trung bình
age_mean = df["Age"].mean()
df["Age"] = df["Age"].fillna(age_mean)  # Không sử dụng inplace=True

# Thay thế giá trị thiếu ở cột "Weight" bằng giá trị trung bình
weight_mean = df["Weight"].mean()
df["Weight"] = df["Weight"].fillna(weight_mean)  # Không sử dụng inplace=True

# Loại bỏ các dòng có giá trị thiếu ở cả hai cột "Age" và "Weight"
df = df.dropna(subset=["Age", "Weight"], how="all")  # Không sử dụng inplace=True


In [14]:
#8️ Xử lý giá trị thiếu trên HR5
def fill_missing_hr(row, col):
    hr_cols = ["HR1", "HR2", "HR3", "HR4", "HR5", "HR6"]
    idx = hr_cols.index(col)
    prev_value = row[hr_cols[idx - 1]] if idx > 0 else np.nan
    next_value = row[hr_cols[idx + 1]] if idx < len(hr_cols) - 1 else np.nan

    # Convert prev_value and next_value to numeric, handling errors
    try:
        prev_value = float(prev_value)
    except (ValueError, TypeError):
        prev_value = np.nan
    try:
        next_value = float(next_value)
    except (ValueError, TypeError):
        next_value = np.nan

    if pd.notna(prev_value) and pd.notna(next_value):
        return (prev_value + next_value) / 2
    elif pd.notna(prev_value):
        return prev_value
    elif pd.notna(next_value):
        return next_value
#8️ Xử lý giá trị thiếu trên HR5
def fill_missing_hr(row, col):
    hr_cols = ["HR1", "HR2", "HR3", "HR4", "HR5", "HR6"]
    idx = hr_cols.index(col)
    prev_value = row[hr_cols[idx - 1]] if idx > 0 else np.nan
    next_value = row[hr_cols[idx + 1]] if idx < len(hr_cols) - 1 else np.nan

    # Convert prev_value and next_value to numeric, handling errors
    try:
        prev_value = float(prev_value)
    except (ValueError, TypeError):
        prev_value = np.nan
    try:
        next_value = float(next_value)
    except (ValueError, TypeError):
        next_value = np.nan

    if pd.notna(prev_value) and pd.notna(next_value):
        return (prev_value + next_value) / 2
    elif pd.notna(prev_value):
        return prev_value
    elif pd.notna(next_value):
        return next_value
    elif pd.notna(row[hr_cols].mean()):
        return row[hr_cols].mean()
    else:
        return np.nan

In [15]:
# Reindex lại dữ liệu
df.reset_index(drop=True, inplace=True)

df.head()


,ID,Age,Weight,HR1,HR2,HR3,HR4,HR5,HR6,Firstname,Lastname
0,1.0,56.000000,70.000000,72,69,71,-,-,-,Micky,Mous
1,2.0,34.000000,70.260000,-,-,-,85,84,76,Donald,Duck
2,3.0,16.000000,72.770909,-,-,-,65,69,72,Mini,Mouse
3,4.0,37.545455,78.000000,78,79,72,-,-,-,Scrooge,McDuck
4,5.0,54.000000,90.110000,-,-,-,69,NaN,75,Pink,Panther
